In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
from keras.utils import np_utils

import cv2
import numpy as np

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5103)
/Library/Python/2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
catmapfilename='breed_map.list'
catmapfile = open(catmapfilename,'r')
categoryMap = dict()
for line in catmapfile.readlines():
    ls = line.split()
    category = ls[0]
    breed = ls[1]
    categoryMap[category] = breed

categoryIndex = dict(zip(categoryMap.keys(),range(len(categoryMap))))
nb_classes = len(categoryMap)

In [3]:
input_photos = []
breedlist = []
posfilename = "positives_shuf.txt"
inputdirname = "CroppedDogs/"
posfile = open(posfilename,'r')
for line in posfile.readlines():
    infile = line.strip()
    ls0 = infile.split("/")
    ls1 = ls0[1].split("_")
    category = ls1[0]
    img = cv2.resize(cv2.imread(infile), (299, 299), cv2.INTER_LINEAR)
    input_photos.append(img)
    breedlist.append(categoryIndex[category])

In [4]:
print len(input_photos),len(breedlist)
batch_size = 10
nb_epoch = 10

22126 22126


In [5]:
train = np.array(input_photos)
labels = np.array(breedlist)

print train.shape

nToUse = len(train)
testFraction = 1.0/6.0
nTest = int(testFraction*nToUse)

X_test = train[:nTest]
y_test = labels[:nTest]

X_train = train[nTest:]
y_train = labels[nTest:]

data_augmentation = False

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# input image dimensions
img_rows, img_cols = X_train.shape[1], X_train.shape[2]

if K.image_dim_ordering() == 'th':
    X_train = np.swapaxes(X_train,1,3)
    X_train = np.swapaxes(X_train,2,3)
    X_test = np.swapaxes(X_test,1,3)
    X_test = np.swapaxes(X_test,2,3)
    input_shape = (train.shape[3], img_rows, img_cols)
else:
	input_shape = (img_rows, img_cols, train.shape[3])

print input_shape

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

(22126, 299, 299, 3)
(3, 299, 299)


In [6]:
# create the base pre-trained model

# this could also be the output a different Keras model or layer
#input_tensor = Input(shape=(3, 128, 128))  # this assumes K.image_dim_ordering() == 'tf'

base_model = InceptionV3(weights='imagenet', include_top=False)

print len(base_model.layers)
base_model.layers.pop()

# add a global spatial average pooling layer
x = base_model.output

x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(nb_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

217


In [7]:
model.compile(optimizer='adadelta', loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_split=1/10.0)

Train on 16595 samples, validate on 1844 samples
Epoch 1/10
16595/16595 [==============================] - 2119s - loss: 2.7533 - acc: 0.3968 - val_loss: 8.0533 - val_acc: 0.0087

In [9]:
model.save("dog_cnn_inception.keras")

In [8]:
score = model.evaluate(X_test, Y_test, verbose=0)
Y_pred = model.predict(X_test)
print 'Test score:', score[0]
print 'Test accuracy:', score[1]


MemoryError: Error allocating 177020928 bytes of device memory (out of memory).
Apply node that caused the error: GpuElemwise{Composite{(((i0 - i1) * i2) + i3)},no_inplace}(GpuElemwise{mul,no_inplace}.0, GpuReshape{4}.0, GpuElemwise{Composite{(i0 / sqrt(clip((i1 + i2), i3, i4)))},no_inplace}.0, GpuReshape{4}.0)
Toposort index: 2803
Inputs types: [CudaNdarrayType(float32, 4D), CudaNdarrayType(float32, (True, False, True, True)), CudaNdarrayType(float32, (True, False, True, True)), CudaNdarrayType(float32, (True, False, True, True))]
Inputs shapes: [(32, 64, 147, 147), (1, 64, 1, 1), (1, 64, 1, 1), (1, 64, 1, 1)]
Inputs strides: [(1382976, 21609, 147, 1), (0, 1, 0, 0), (0, 1, 0, 0), (0, 1, 0, 0)]
Inputs values: ['not shown', 'not shown', 'not shown', 'not shown']
Outputs clients: [[if{inplace,gpu}(keras_learning_phase, GpuElemwise{Composite{(((i0 - i1) * i2) + i3)},no_inplace}.0, GpuElemwise{Composite{(((i0 - i1) * i2) + i3)},no_inplace}.0)]]

HINT: Re-running with most Theano optimizati